In [ ]:
!git clone https://github.com/openai/multiagent-particle-envs.git
!pip install gym==0.10.5

Cloning into 'multiagent-particle-envs'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 242 (delta 0), reused 3 (delta 0), pack-reused 237
Receiving objects: 100% (242/242), 107.24 KiB | 892.00 KiB/s, done.
Resolving deltas: 100% (127/127), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 7.2 MB/s 
     |████████████████████████████████| 1.1 MB 41.3 MB/s 
  Created wheel for gym: filename=gym-0.10.5-py3-none-any.whl size=1581307 sha256=b176d05be43b1156ea7a5bec04b1937e70c98b0f576c6110a4e3431b77dd93c9
  Stored in directory: /root/.cache/pip/wheels/7a/2c/df/a05b548a40fae16ca400ecbeda0067e1a296499c1fbd7e0c9a
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


In [ ]:
cd multiagent-particle-envs

/content/multiagent-particle-envs


In [ ]:
!pip install -e.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/multiagent-particle-envs
  Running setup.py develop for multiagent


In [ ]:
class Buffer:
  def __init__(self):
    self.agent1_reward = []
    self.agent1_value  = []
    self.agent1_loss   = []
    self.agent1_actor  = []
    self.agent1_critic = []
    self.agent1_nextval= []
    self.agent1_message= []
    self.agent1_returns= []
    self.agent2_reward = []
    self.agent2_value  = []
    self.agent2_loss   = []
    self.agent2_actor  = []
    self.agent2_critic = []
    self.agent2_nextval= []
    self.agent2_message= []
    self.agent2_returns= []
    self.agent3_reward = []
    self.agent3_value  = []
    self.agent3_loss   = []
    self.agent3_actor  = []
    self.agent3_critic = []
    self.agent3_nextval= []
    self.agent3_message= []
    self.agent3_returns= []
    self.main_loss     = []
    self.episodes      = []
    self.cout          = []
    self.messages_list = [[],[],[]]
  def agent1_appendning(self,reward,value,loss,actor,critic,next_value,message,returns):
    self.agent1_reward.append(torch.tensor(reward).cpu())
    self.agent1_value.append(torch.tensor(value).cpu())
    self.agent1_loss.append(torch.tensor(loss).cpu())
    self.agent1_actor.append(torch.tensor(actor).cpu())
    self.agent1_critic.append(torch.tensor(critic).cpu())
    self.agent1_nextval.append(torch.tensor(next_value).cpu())
    self.agent1_message.append(torch.tensor(message).cpu())
    self.agent1_returns.append(torch.tensor(returns).cpu())
  def agent2_appendning(self,reward,value,loss,actor,critic,next_value,message,returns):
    self.agent2_reward.append(torch.tensor(reward).cpu())
    self.agent2_value.append(torch.tensor(value).cpu())
    self.agent2_loss.append(torch.tensor(loss).cpu())
    self.agent2_actor.append(torch.tensor(actor).cpu())
    self.agent2_critic.append(torch.tensor(critic).cpu())
    self.agent2_nextval.append(torch.tensor(next_value).cpu())
    self.agent2_message.append(torch.tensor(message).cpu())
    self.agent2_returns.append(torch.tensor(returns).cpu())
  def agent3_appendning(self,reward,value,loss,actor,critic,next_value,message,returns):
    self.agent3_reward.append(torch.tensor(reward).cpu())
    self.agent3_value.append(torch.tensor(value).cpu())
    self.agent3_loss.append(torch.tensor(loss).cpu())
    self.agent3_actor.append(torch.tensor(actor).cpu())
    self.agent3_critic.append(torch.tensor(critic).cpu())
    self.agent3_nextval.append(torch.tensor(next_value).cpu())
    self.agent3_message.append(torch.tensor(message).cpu())
    self.agent3_returns.append(torch.tensor(returns).cpu())


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.autograd import variable as v

class Actor(nn.Module):
  def __init__(self,state_size,action_size,message_size):
    super(Actor,self).__init__()
    self.state_size   = state_size
    self.action_size  = action_size
    self.message_size = message_size
    self.device       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.fc3          = nn.Linear(64,128).to(self.device)
    self.fc4          = nn.Linear(128,256).to(self.device)
    self.fc7          = nn.Linear(256,128).to(self.device)
    self.fc8          = nn.Linear(128,32).to(self.device)
    self.fc9          = nn.Linear(32,self.action_size).to(self.device)
  def forward(self,state,message):
    message           = torch.tensor(message).float().to(self.device)
    self.fc1          = nn.Linear(state.shape[0],32).to(self.device)
    self.fc2          = nn.Linear(message.shape[0],32).to(self.device)
    a1                = self.fc1(state)
    m1                = self.fc2(message)
    cat               = torch.cat([a1,m1],0)
    self.cat          = nn.Linear(cat.shape[0],64).to(self.device)
    cat               = self.cat(cat)
    x                 = self.fc8(f.relu(self.fc7(f.relu(self.fc4(f.relu(self.fc3(cat)))))))
    x                 = self.fc9(x)
    x                 = f.softmax(x)
    return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim

class Critic(nn.Module):
  def __init__(self,state_size,action_size,reward_size):
    super(Critic,self).__init__()
    self.device       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.fc4 = nn.Linear(64,128).to(self.device)
    self.fc5 = nn.Linear(128,128).to(self.device)
    self.fc6 = nn.Linear(128,128).to(self.device)
    self.fc7 = nn.Linear(128,64).to(self.device)
    self.fc8 = nn.Linear(64,1).to(self.device)
  def forward(self,state,action,reward):
    self.fc1 = nn.Linear(state.shape[0],32).to(self.device)
    self.fc2 = nn.Linear(action.shape[0],32).to(self.device)
    self.fc3 = nn.Linear(reward.shape[0],32).to(self.device)
    s        = self.fc1(state)
    a        = self.fc2(action)
    r        = self.fc3(reward)
    cat      = torch.cat([s,a,r],0)
    self.cat = nn.Linear(cat.shape[0],64).to(self.device)
    cat      = self.cat(cat)
    x        = f.relu(self.fc8(f.relu(self.fc7(f.relu(self.fc6(f.relu(self.fc5(f.relu(self.fc4(cat))))))))))
    return x

In [ ]:
class Msgnet(nn.Module):
  def __init__(self):
    super(Msgnet,self).__init__()
    self.pa1    = nn.Linear(64,128)
    self.pa2    = nn.Linear(128,128)
    self.pa3    = nn.Linear(128,128)
    self.pa4    = nn.Linear(256,128)
    self.pa5    = nn.Linear(256,128)
    self.pa6    = nn.Linear(128,128)
    self.pa7    = nn.Linear(128,128)
    self.pa8    = nn.Linear(32,128)
    self.pa9    = nn.Linear(32,128)
    self.pa10   = nn.Linear(5,128)
    self.value  = nn.Linear(1,4)
    self.lstm   = nn.Linear(4,32)
    self.l1     = nn.Linear(32,128)
    self.l2     = nn.Linear(128,64)
    self.l3     = nn.Linear(64,32)
    self.l4     = nn.Linear(32,1)
  def forward(self,parameter,value,messages):
    #prsent
    x0          = self.pa1(parameter[0]).reshape((-1,))
    x1          = self.pa2(parameter[1]).reshape((-1,))
    x2          = self.pa3(parameter[2]).reshape((-1,))
    x3          = self.pa4(parameter[3]).reshape((-1,))
    x4          = self.pa5(parameter[4]).reshape((-1,))
    x5          = self.pa6(parameter[5]).reshape((-1,))
    x6          = self.pa7(parameter[6]).reshape((-1,))
    x7          = self.pa8(parameter[7]).reshape((-1,))
    x8          = self.pa9(parameter[8]).reshape((-1,))
    x9          = self.pa10(parameter[9]).reshape((-1,))
    k           = torch.cat([x0,x1,x2,x3,x4,x5,x6,x7,x8,x9])
    k           = torch.reshape(k,(17728,4))
    q           = self.value(value)
    mat1        = torch.matmul(k,q)
    mat1        = torch.reshape(mat1,(4432,4)).float()
    mat2        = torch.matmul(mat1,messages.float())
    mat2        = torch.reshape(mat2,(1108,4))
    w           = f.relu(self.l4(f.relu(self.l3(f.relu(self.l2(f.relu(self.l1(f.relu(self.lstm(mat2))))))))))
    w           = f.softmax(w,dim = 0)
    w           = w.mean()
    return w

In [ ]:
class Agent1:
  def __init__(self,state_size,action_size,reward_size,buffer):
    self.state_size = state_size
    self.action_size = action_size
    self.reward_size = reward_size 
    self.buffer      = buffer
    self.device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.lr1         = 0.00000001
    self.lr2         = 0.00000009
    self.lr3         = 0.0000007
    self.gamma       = 0.99
    self.lamda       = 0.95
    self.actor       = Actor(self.state_size,self.action_size,self.reward_size).to(self.device)
    self.critic      = Critic(self.state_size,self.action_size,self.reward_size).to(self.device)
    self.msgnet      = Msgnet().to(self.device)
    self.actor_optim = optim.Adam(self.actor.parameters() ,lr = self.lr1)
    self.critic_optim= optim.Adam(self.critic.parameters(),lr = self.lr2)
    self.msg_optim   = optim.Adam(self.msgnet.parameters(),lr = self.lr3)
    self.policy      = []
    self.values      = []
  def choose_action(self,state,message):
    state = torch.tensor(state).float().to(self.device)
    act   = self.actor(state,message)
    return act
  def q_value(self,state,action,reward):
    reward = torch.tensor([reward]).float().to(self.device)
    state  = torch.tensor(state).float().to(self.device)
    value  = self.critic(state,action,reward)
    return value
  def ppo_iter(self,reward,value,next_value,done):
    gae    = 0
    returns= []
    for i in range(5):
      delta = reward + self.gamma*(1-done)*next_value - value
      gae   = delta + self.lamda*(1-done)*gae
      returns.insert(0,gae + value+next_value)
    return returns
  def concate(self,messages,i):
    m1  = torch.cat([messages[0],self.buffer.agent2_value[i]])
    m2  = torch.cat([messages[1],self.buffer.agent3_value[i]])
    msg = torch.cat([m1,m2])
    return msg
  def message(self,value,messages,i):
    oth_messages = self.concate(messages,i)
    value        = value
    parameter    = list(self.actor.parameters())
    msg          = self.msgnet(parameter,value,oth_messages)
    return msg
  def learn(self,state,next_state,reward,done,next_value,i,messages):
    action        = self.choose_action(state,self.concate(messages,i))
    value         = self.q_value(state,action,reward)
    msg           = self.message(value,messages,i)
    returns       = torch.tensor(self.ppo_iter(reward,value,next_value,done))
    advantage     = returns - value
    self.policy.append(action)
    self.values.append(value)
    idx           = torch.argmax(torch.tensor(self.values))
    idx           = idx.item()
    prev_policy   = self.policy[idx]
    loss          = (prev_policy-action)**2*value*advantage
    loss          = torch.tensor(loss).to(self.device)
    loss.requires_grad = True
    critic_loss   = (returns - value)**2
    critic_loss   = torch.tensor(critic_loss.mean()).to(self.device)
    critic_loss.requires_grad = True
    msg_loss      = loss+0.5*critic_loss
    msg_loss      = torch.tensor([msg_loss.mean()])
    #msg_loss.requires_grad = True
    #==========================================================================================================================================================
    self.actor_optim.zero_grad()
    loss.backward()
    self.actor_optim.step()
    self.critic_optim.zero_grad()
    critic_loss.backward()
    self.critic_optim.step()
    self.msg_optim.zero_grad()
    msg_loss.backward()
    self.msg_optim.step()
    #==========================================================================================================================================================
    return msg

In [ ]:
torch.set_grad_enabled(True)
class Agent2:
  def __init__(self,state_size,action_size,reward_size,buffer):
    self.state_size = state_size
    self.action_size = action_size
    self.reward_size = reward_size 
    self.buffer      = buffer
    self.device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.lr1         = 0.00000001
    self.lr2         = 0.00000009
    self.lr3         = 0.0000007
    self.gamma       = 0.99
    self.lamda       = 0.95
    self.actor       = Actor(self.state_size,self.action_size,self.reward_size).to(self.device)
    self.critic      = Critic(self.state_size,self.action_size,self.reward_size).to(self.device)
    self.msgnet      = Msgnet().to(self.device)
    self.actor_optim = optim.Adam(self.actor.parameters() ,lr = self.lr1)
    self.critic_optim= optim.Adam(self.critic.parameters(),lr = self.lr2)
    self.msg_optim   = optim.Adam(self.msgnet.parameters(),lr = self.lr3)
    self.policy      = []
    self.values       = []
  def choose_action(self,state,message):
    state = torch.tensor(state).float().to(self.device)
    act   = self.actor(state,message)
    return act
  def q_value(self,state,action,reward):
    reward = torch.tensor([reward]).float().to(self.device)
    state  = torch.tensor(state).float().to(self.device)
    value  = self.critic(state,action,reward)
    return value
  def ppo_iter(self,reward,value,next_value,done):
    gae    = 0
    returns= []
    for i in range(5):
      delta = reward + self.gamma*(1-done)*next_value - value
      gae   = delta + self.lamda*(1-done)*gae
      returns.insert(0,gae + value+next_value)
    return returns
  def concate(self,messages,i):
    m1  = torch.tensor([messages[0]])
    m2  = torch.tensor([messages[1]])
    v1  = torch.tensor(self.buffer.agent3_value[i])
    v2  = torch.tensor(self.buffer.agent1_value[i])
    m1  = torch.cat([m1,v1])
    m2  = torch.cat([m2,v2])
    msg = torch.cat([m1,m2])
    return msg
  def message(self,value,messages,i):
    oth_messages = self.concate(messages,i)
    value        = value
    parameter    = list(self.actor.parameters())
    msg          = self.msgnet(parameter,value,oth_messages)
    return msg
  def learn(self,state,next_state,reward,done,next_value,i,messages):
    action        = self.choose_action(state,self.concate(messages,i))
    value         = self.q_value(state,action,reward)
    msg           = self.message(value,messages,i)
    returns       = torch.tensor(self.ppo_iter(reward,value,next_value,done))
    advantage     = returns - value
    self.policy.append(action)
    self.values.append(value)
    self.buffer.agent2_value.append([value])
    idx           = torch.argmax(torch.tensor(self.values))
    idx           = idx.item()
    prev_policy   = self.policy[idx]
    loss          = (prev_policy-action)**2*value*advantage
    loss          = torch.tensor([loss.mean()]).to(self.device)
    loss.requires_grad = True
    critic_loss   = (returns - value)**2
    critic_loss   = torch.tensor(critic_loss.mean()).to(self.device)
    critic_loss.requires_grad = True
    msg_loss      = loss+0.5*critic_loss
    msg_loss      = torch.tensor(msg_loss.mean())
    msg_loss.requires_grad = True
    msg_loss.retain_grad()
    #==========================================================================================================================================================
    self.actor_optim.zero_grad()
    loss.backward()
    self.actor_optim.step()
    self.critic_optim.zero_grad()
    critic_loss.backward()
    self.critic_optim.step()
    self.msg_optim.zero_grad()
    msg_loss.backward()
    self.msg_optim.step()
    #==========================================================================================================================================================
    return msg

In [ ]:
class Agent3:
  def __init__(self,state_size,action_size,reward_size,buffer):
    self.state_size = state_size
    self.action_size = action_size
    self.reward_size = reward_size 
    self.buffer      = buffer
    self.device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.lr1         = 0.00000001
    self.lr2         = 0.00000009
    self.lr3         = 0.0000007
    self.gamma       = 0.99
    self.lamda       = 0.95
    self.actor       = Actor(self.state_size,self.action_size,self.reward_size).to(self.device)
    self.critic      = Critic(self.state_size,self.action_size,self.reward_size).to(self.device)
    self.msgnet      = Msgnet().to(self.device)
    self.actor_optim = optim.Adam(self.actor.parameters() ,lr = self.lr1)
    self.critic_optim= optim.Adam(self.critic.parameters(),lr = self.lr2)
    self.msg_optim   = optim.Adam(self.msgnet.parameters(),lr = self.lr3)
    self.policy      = []
    self.values       = []
  def choose_action(self,state,message):
    state = torch.tensor(state).float().to(self.device)
    act   = self.actor(state,message)
    return act
  def q_value(self,state,action,reward):
    reward = torch.tensor([reward]).float().to(self.device)
    state  = torch.tensor(state).float().to(self.device)
    value  = self.critic(state,action,reward)
    return value
  def ppo_iter(self,reward,value,next_value,done):
    gae    = 0
    returns= []
    for i in range(5):
      delta = reward + self.gamma*(1-done)*next_value - value
      gae   = delta + self.lamda*(1-done)*gae
      returns.insert(0,gae + value+next_value)
    return returns
  def concate(self,messages,i):
    m1  = torch.tensor([messages[0]])
    m2  = torch.tensor([messages[1]])
    v1  = torch.tensor(self.buffer.agent1_value[i])
    v2  = torch.tensor(self.buffer.agent2_value[i])
    m1  = torch.cat([m1,v1])
    m2  = torch.cat([m2,v2])
    msg = torch.cat([m1,m2])
    return msg
  def message(self,value,messages,i):
    oth_messages = self.concate(messages,i)
    value        = value
    parameter    = list(self.actor.parameters())
    msg          = self.msgnet(parameter,value,oth_messages)
    return msg
  def learn(self,state,next_state,reward,done,next_value,i,messages):
    action        = self.choose_action(state,self.concate(messages,i))
    value         = self.q_value(state,action,reward)
    msg           = self.message(value,messages,i)
    returns       = torch.tensor(self.ppo_iter(reward,value,next_value,done))
    advantage     = returns - value
    self.policy.append(action)
    self.values.append(value)
    self.buffer.agent3_value.append([value])
    idx           = torch.argmax(torch.tensor(self.values))
    idx           = idx.item()
    prev_policy   = self.policy[idx]
    loss          = (prev_policy-action)**2*value*advantage
    loss          = torch.tensor([loss.mean()]).to(self.device)
    loss.requires_grad = True
    critic_loss   = (returns - value)**2
    critic_loss   = torch.tensor(critic_loss.mean()).to(self.device)
    critic_loss.requires_grad = True
    msg_loss      = loss+0.5*critic_loss
    msg_loss      = torch.tensor(msg_loss.mean())
    msg_loss.requires_grad = True
    msg_loss.retain_grad()
    #==========================================================================================================================================================
    self.actor_optim.zero_grad()
    loss.backward()
    self.actor_optim.step()
    self.critic_optim.zero_grad()
    critic_loss.backward()
    self.critic_optim.step()
    self.msg_optim.zero_grad()
    msg_loss.backward()
    self.msg_optim.step()
    #==========================================================================================================================================================
    return msg

In [ ]:
torch.set_grad_enabled(True)
class Main:
  def __init__(self,state_size,action_size,reward_size,buffer,n_games,steps):
    self.state_size = state_size
    self.action_size = action_size
    self.reward_size = reward_size
    self.n_games     = n_games
    self.steps       = steps
    self.gamma       = 0.99
    self.buffer      = buffer
    self.device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.critic      = Critic(self.state_size,self.action_size,self.reward_size).to(self.device)
    self.optim       = optim.Adam(self.critic.parameters(),lr = 0.000005)
    self.agent1      = Agent1(self.state_size,self.action_size,self.reward_size,self.buffer)
    self.agent2      = Agent2(self.state_size,self.action_size,self.reward_size,self.buffer)
    self.agent3      = Agent3(self.state_size,self.action_size,self.reward_size,self.buffer)
  def concate(self,messages,i):
    value1  = torch.tensor(self.buffer.agent1_value[i])
    value2  = torch.tensor(self.buffer.agent2_value[i])
    value3  = torch.tensor(self.buffer.agent3_value[i])
    msg1,msg2,msg3    = torch.tensor([messages[0]]),torch.tensor([messages[1]]),torch.tensor([messages[2]])
    msg1    = torch.cat([msg1,value1])
    msg2    = torch.cat([msg2,value2])
    msg3    = torch.cat([msg3,value3])
    amsg1 = torch.cat([msg2,msg3])
    amsg2 = torch.cat([msg3,msg1])
    amsg3 = torch.cat([msg1,msg2])
    return amsg1,amsg2,amsg3
  def choose_actions(self,state,messages,i):
    msg1,msg2,msg3 = self.concate(messages,i)
    actions = []
    act1    = self.agent1.choose_action(state[0],msg1)
    act2    = self.agent2.choose_action(state[1],msg2)
    act3    = self.agent3.choose_action(state[2],msg3)
    actions.append(act1)
    actions.append(act2)
    actions.append(act3)
    return actions
  def loss(self,reward,done,next_values,values):
    gae     = 0
    returns = []
    for i in range(1):
      delta = sum(reward) + (1-sum(done))*0.99*sum(next_values)- sum(values)
      gae   = delta + 0.95*gae*(1-sum(done))
      returns.insert(0,gae+sum(next_values))
    returns = torch.tensor([returns]).float().to(self.device)
    return returns
  def next_values(self,next_state,action,reward):
    val  = []
    for i in range(3):
      rewards  = torch.tensor([reward[i]]).float().to(self.device)
      obs = torch.tensor(next_state[i]).float().to(self.device)
      value = self.critic(obs,action[i],rewards)
      val.append(value)
    return val
  def update(self,state,next_state,rewards,messages,done,i):
    actions  = self.choose_actions(state,messages,i)
    values   = self.next_values(state,actions,rewards)
    next_act = self.choose_actions(next_state,messages,i)
    nvalues  = self.next_values(next_state,next_act,rewards)
    loss     = self.loss(rewards,done,nvalues,values).mean()
    loss.requires_grad = True
    msg1     = self.agent1.learn(state[0],next_state[0],rewards[0],done[0],nvalues[0],i,[messages[1],messages[2]])
    msg2     = self.agent2.learn(state[1],next_state[1],rewards[1],done[1],nvalues[1],i,[messages[2],messages[0]])
    msg3     = self.agent3.learn(state[2],next_state[2],rewards[2],done[2],nvalues[2],i,[messages[0],messages[1]])
    self.buffer.messages_list[0].append(msg1)
    self.buffer.messages_list[1].append(msg2)
    self.buffer.messages_list[2].append(msg3)
    print(len(self.buffer.messages_list[0]))
    self.optim.zero_grad()
    loss.backward()
    self.optim.step()
  def run(self,env,idx):
    for i in range(self.n_games):
      state = env.reset()
      score = [0,0,0]
      done  = [False*3]
      for step in range(self.steps):  
        if i==0 and step==0:
          messages = [torch.tensor([1]),torch.tensor([1]),torch.tensor([1])]
        else:
          print(self.buffer.messages_list[0])
          msg1 = self.buffer.messages_list[0][idx]
          msg2 = self.buffer.messages_list[1][idx]
          msg3 = self.buffer.messages_list[2][idx]
          messages=[msg1,msg2,msg3]
        actions = self.choose_actions(state,messages,idx)
        next_state,reward,done,infor = env.step(actions)
        if done:
          self.update(state,next_state,reward,messages,done,idx)
          state = next_state
          idx  = idx+1
        else:
          print("completed")

In [ ]:
if __name__ =="__main__":
  buffer = Buffer()
  from make_env import make_env
  buffer.agent1_value.append(torch.tensor([1]))
  buffer.agent2_value.append(torch.tensor([1]))
  buffer.agent3_value.append(torch.tensor([1]))
  env = make_env("simple_adversary")
  m = Main(21,5,5,buffer,10,10)
  cout = -1
  m.run(env,cout)